In [1]:
!pip install git+https://github.com/microsoft/recommenders.git

  Cloning https://github.com/microsoft/recommenders.git to /tmp/pip-req-build-6huvc31e
  Running command git clone --filter=blob:none --quiet https://github.com/microsoft/recommenders.git /tmp/pip-req-build-6huvc31e
  Resolved https://github.com/microsoft/recommenders.git to commit 0aaa80b4ec029a20c99abb6fcf3b7efe42e57eb8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 43.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 84.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 20.7 MB/s eta 0:00:00
  Installing bu

In [2]:
import os
import sys
import cornac
import pandas as pd

from recommenders.datasets.python_splitters import python_random_split
from recommenders.evaluation.python_evaluation import map, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.models.cornac.cornac_utils import predict_ranking
from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED

from cornac import Experiment
from cornac.eval_methods import RatioSplit
from cornac.metrics import Recall, NDCG, NCRR

from cornac.hyperopt import GridSearch,Discrete
from cornac.models import WMF
import torch

print(f"System version: {sys.version}")
print(f"Cornac version: {cornac.__version__}")

System version: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
Cornac version: 1.18


In [3]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')
train_path = '/content/drive/MyDrive/SMU_MITB_RS/Project1/cs608_ip_train_v3.csv'
test_path =  '/content/drive/MyDrive/SMU_MITB_RS/Project1/cs608_ip_probe_v3.csv'

import pandas as pd

# Read the file into a pandas DataFrame
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
# Combine the two df for ratiosplit
df = pd.concat([df_train, df_test], axis=0, ignore_index=True)

# Display the DataFrame
print(df)

Mounted at /content/drive
        user_id  item_id  rating
0         12108        1       2
1         12108        2       5
2         12108        3       4
3         12108        4       4
4         12108        5       5
...         ...      ...     ...
281316    12568     1511       3
281317    12568    12389       4
281318    14361    18535       5
281319    14361    16572       5
281320    14361    19047       5

[281321 rows x 3 columns]


In [4]:
print(df.dtypes)

user_id    int64
item_id    int64
rating     int64
dtype: object


In [5]:
train_set = cornac.data.Dataset.from_uir(df.itertuples(index=False))

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))

Number of users: 21124
Number of items: 31203


In [ ]:
# code for GridSearch, since the time costing, did not apply on WRMF models

ratio_split = RatioSplit(data=df.values, test_size=0.1, val_size=0.1, rating_threshold=4.0, seed=2024)
param_grid =[
    Discrete(name = 'k',values =[50, 100] ),
    Discrete(name = 'max_iter',values=[50,100,200] ),
    Discrete(name = 'learning_rate',values =[0.001,0.01,0.1]),
    Discrete(name = 'lambda_u',values = [0.001,0.01,0.1]),
    Discrete(name = 'lambda_v',values = [0.001,0.01,0.1]),
]

WMF = cornac.models.WMF(
    k=50,
    max_iter=200,
    learning_rate=0.05,
    lambda_reg=0.001,
    verbose=True,
    seed=SEED
)

# Initialize GridSearch
gs= GridSearch(model=WMF,space = param_grid,metric = Recall(k=50),eval_method = ratio_split)

# Run the grid search
cornac.Experiment(
    eval_method=ratio_split,
    models=[gs],
    metrics = [NDCG(k=50),NCRR(k=50),Recall(k=50)] , # 添加 NDCG 指标
    user_based=False
).run()

print("Best parameters found:", gs.best_params)


Evaluating: {'k': 50, 'lambda_reg': 0.001, 'learning_rate': 0.001, 'max_iter': 50}


  0%|          | 0/50 [00:00<?, ?it/s]

Optimization finished!
Evaluating: {'k': 50, 'lambda_reg': 0.001, 'learning_rate': 0.001, 'max_iter': 100}


  0%|          | 0/100 [00:00<?, ?it/s]

Optimization finished!
Evaluating: {'k': 50, 'lambda_reg': 0.001, 'learning_rate': 0.001, 'max_iter': 200}


...

  0%|          | 0/100 [00:00<?, ?it/s]

Optimization finished!
Evaluating: {'k': 100, 'lambda_reg': 0.1, 'learning_rate': 0.1, 'max_iter': 200}


  0%|          | 0/200 [00:00<?, ?it/s]

Optimization finished!
Best parameter settings: {'k': 100, 'lambda_reg': 0.01, 'learning_rate': 0.1, 'max_iter': 200}
Recall@50 = 0.1213

VALIDATION:
...
               | NCRR@50 | NDCG@50 | Recall@50 | Time (s)
-------------- + ------- + ------- + --------- + --------
GridSearch_BPR |  0.0199 |  0.0415 |    0.1213 |  74.2757

TEST:
...
               | NCRR@50 | NDCG@50 | Recall@50 | Train (s) | Test (s)
-------------- + ------- + ------- + --------- + --------- + --------
GridSearch_BPR |  0.0196 |  0.0416 |    0.1228 | 4684.1858 |  80.1860

Best parameters found: {'k': 100, 'lambda_reg': 0.01, 'learning_rate': 0.1, 'max_iter': 200}


AttributeError: 'GridSearch' object has no attribute 'best_result'

In [6]:
# 7

# build the model
WMF = cornac.models.WMF(
    k=100,
    max_iter=100,
    a = 1.00,
    b = 0.1,
    learning_rate=0.001,
    lambda_u= 0.01,
    lambda_v = 0.01,
    verbose=True,
    seed=2024
)

# run the model
ratio_split = RatioSplit(data=df.values, rating_threshold= 4.0,test_size=0.2, seed=2024)
metrics = [NDCG(k=50),NCRR(k=50),Recall(k=50)]

experiment = Experiment(eval_method=ratio_split,models=[WMF], metrics=metrics)
experiment.run()

  0%|          | 0/100 [00:00<?, ?it/s]

Learning completed!

TEST:
...
    | NCRR@50 | NDCG@50 | Recall@50 | Train (s) | Test (s)
--- + ------- + ------- + --------- + --------- + --------
WMF |  0.0545 |  0.0724 |    0.1329 |  664.5766 |  98.2012



In [ ]:
import numpy as np
# Retrieve the top 50 recommendations for each user
train_user_ids = ratio_split.train_set.user_ids
test_user_ids = ratio_split.test_set.user_ids


all_user_ids = np.unique(np.concatenate((train_user_ids, test_user_ids)))

# sort by user_ids
sorted_user_ids = np.sort(all_user_ids)

# write the result into txt
output_file_path = 'all_user_top_50_recommendations_sorted.txt'
with open(output_file_path, 'w') as file:
    for user_id in sorted_user_ids:
        item_recommendations = WMF.recommend(user_id, k=50)
        recommendation_line = ' '.join(str(item_id) for item_id in item_recommendations)
        file.write(recommendation_line + '\n')



In [ ]:
print(sorted_user_ids.shape)

(21124,)


In [ ]:
import os
print(os.getcwd())


/content


In [ ]:
!ls


all_user_top_50_recommendations_sorted.txt  CornacExp-2024-05-18_07-00-52-024384.log  sample_data
CornacExp-2024-05-18_06-20-14-050777.log    CornacExp-2024-05-18_07-40-24-222166.log
CornacExp-2024-05-18_06-39-40-892193.log    drive


In [ ]:
from google.colab import files
files.download('all_user_top_50_recommendations_sorted.txt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>